In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

!nvidia-smi

# Load training and test datasets from Google Drive
train_df = pd.read_csv('/content/drive/MyDrive/dataset/train2 (1).csv', encoding='utf-8', encoding_errors='ignore')
test_df = pd.read_csv('/content/drive/MyDrive/dataset/test2 (1).csv', encoding='utf-8', encoding_errors='ignore')

print('Number of training sentences: {:,}'.format(train_df.shape[0]))
print('Number of test sentences: {:,}'.format(test_df.shape[0]))

# Display first few rows
print("\nTraining data:")
print(train_df.head(3))
print("\nTest data:")
print(test_df.head(3))

# Rename columns to standardize
train_df.rename(columns={'Sentiment':'label', 'Data':'Text'}, inplace=True)
test_df.rename(columns={'Sentiment':'label', 'Data':'Text'}, inplace=True)

# Keep only necessary columns
train_df = train_df[['label', 'Text']]
test_df = test_df[['label', 'Text']]

print(f"\nTraining data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")

# Check label distribution
print("\nTraining set label distribution:")
print(train_df.label.value_counts())
print("\nTest set label distribution:")
print(test_df.label.value_counts())

# Plot training label distribution
train_df.label.value_counts().plot(kind='bar', title='Training Set Label Distribution')
plt.show()

import warnings
warnings.filterwarnings("ignore")

# Preprocessing patterns
urlPattern = r"((http://)[^ ]*|(https://)[^ ]*|(www\.)[^ ]*)"
userPattern = '@[^\s]+'
hashtagPattern = '#[^\s]+'
sequencePattern = r"(.)\1\1+"
seqReplacePattern = r"\1\1"

def preprocess_apply(tweet):
    tweet = tweet.lower()
    # Replace all URls with ''
    tweet = re.sub(urlPattern,'',tweet)
    # Replace @USERNAME to ''.
    tweet = re.sub(userPattern,'', tweet)
    # Replace 3 or more consecutive letters by 2 letter.
    tweet = re.sub(sequencePattern, seqReplacePattern, tweet)
    # Adding space on either side of '/' to seperate words (After replacing URLS).
    tweet = re.sub(r'/', ' / ', tweet)
    return tweet

from tqdm import tqdm
tqdm.pandas()

# Add word count for analysis
train_df['word_count'] = train_df['Text'].apply(lambda x: len(str(x).split()))
test_df['word_count'] = test_df['Text'].apply(lambda x: len(str(x).split()))

# Plot word count distribution
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.violinplot(x=train_df['word_count'])
plt.title('Training Set: Word Count Distribution')
plt.xlabel('Word Count')

plt.subplot(1, 2, 2)
sns.violinplot(x=test_df['word_count'])
plt.title('Test Set: Word Count Distribution')
plt.xlabel('Word Count')
plt.tight_layout()
plt.show()

print("Training set word count statistics:")
print(train_df.word_count.describe())
print("\nTest set word count statistics:")
print(test_df.word_count.describe())

# Install normalizer
!pip install git+https://github.com/csebuetnlp/normalizer

from transformers import AutoModelForPreTraining, AutoTokenizer
from normalizer import normalize
import torch

# Load the BERT tokenizer
print('Loading BERT tokenizer...')
model_path = "csebuetnlp/banglabert_large"
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Normalize text data
print("Normalizing training data...")
train_df['Text'] = train_df['Text'].progress_apply(normalize)
print("Normalizing test data...")
test_df['Text'] = test_df['Text'].progress_apply(normalize)

print("Normalization completed!")
print("\nTraining data after normalization:")
print(train_df.head())
print("\nTest data after normalization:")
print(test_df.head())

# Create validation split from training data
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(
    train_df,
    test_size=0.2,
    random_state=42,
    stratify=train_df['label']
)

print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")
print(f"Test samples: {len(test_df)}")

# Verify class distribution
print("\nTraining set class distribution:")
print(train_data['label'].value_counts())
print("\nValidation set class distribution:")
print(val_data['label'].value_counts())
print("\nTest set class distribution:")
print(test_df['label'].value_counts())

# Extract sentences and labels
train_sentences = train_data.Text.values
train_labels = train_data.label.values

val_sentences = val_data.Text.values
val_labels = val_data.label.values

test_sentences = test_df.Text.values
test_labels = test_df.label.values

print(f"Training: {len(train_sentences)} sentences, {len(train_labels)} labels")
print(f"Validation: {len(val_sentences)} sentences, {len(val_labels)} labels")
print(f"Test: {len(test_sentences)} sentences, {len(test_labels)} labels")

# Test tokenization
print('\nTokenization example:')
print('Original:', test_sentences[0])
print('Tokenized:', tokenizer.tokenize(test_sentences[0]))
print('Token IDs:', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(test_sentences[0])))

# Example with Bangla text
demo_text = 'আমি তোমাকে ভালবাসি।'
print(f'\nDemo - Original: {demo_text}')
print('Tokenized:', tokenizer.tokenize(demo_text))
print('Token IDs:', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(demo_text)))

# Label mapping - Updated to positive/negative
label_map = {
    0: 'positive',
    1: 'negative'
}

max_len = 128

# Tokenize training data
print("Tokenizing training data...")
input_ids = []
attention_masks = []

for sent in tqdm(train_sentences):
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens=True,
                        max_length=max_len,
                        padding='max_length',
                        truncation=True,
                        return_attention_mask=True,
                        return_tensors='pt',
                   )

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
train_labels = torch.tensor(train_labels)

# Tokenize validation data
print("Tokenizing validation data...")
val_input_ids = []
val_attention_masks = []

for sent in tqdm(val_sentences):
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens=True,
                        max_length=max_len,
                        padding='max_length',
                        truncation=True,
                        return_attention_mask=True,
                        return_tensors='pt',
                   )

    val_input_ids.append(encoded_dict['input_ids'])
    val_attention_masks.append(encoded_dict['attention_mask'])

val_input_ids = torch.cat(val_input_ids, dim=0)
val_attention_masks = torch.cat(val_attention_masks, dim=0)
val_labels = torch.tensor(val_labels)

# Tokenize test data
print("Tokenizing test data...")
test_input_ids = []
test_attention_masks = []

for sent in tqdm(test_sentences):
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens=True,
                        max_length=max_len,
                        padding='max_length',
                        truncation=True,
                        return_attention_mask=True,
                        return_tensors='pt',
                   )

    test_input_ids.append(encoded_dict['input_ids'])
    test_attention_masks.append(encoded_dict['attention_mask'])

test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)
test_labels = torch.tensor(test_labels)

print("Tokenization completed!")

# Create datasets
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

train_dataset = TensorDataset(input_ids, attention_masks, train_labels)
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)
test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)

# Create data loaders
batch_size = 16  # Reduced batch size for better memory management

train_dataloader = DataLoader(
    train_dataset,
    sampler=RandomSampler(train_dataset),
    batch_size=batch_size
)

validation_dataloader = DataLoader(
    val_dataset,
    sampler=SequentialSampler(val_dataset),
    batch_size=batch_size
)

test_dataloader = DataLoader(
    test_dataset,
    sampler=SequentialSampler(test_dataset),
    batch_size=batch_size
)

print(f"Training batches: {len(train_dataloader)}")
print(f"Validation batches: {len(validation_dataloader)}")
print(f"Test batches: {len(test_dataloader)}")

# Custom BanglaBERT with CNN Model
import torch
import torch.nn as nn
from transformers import AutoModel, AutoConfig
import os

class CustomBanglaBertWithCNN(nn.Module):
    def __init__(self, model_name=model_path, num_labels=2, num_filters=256, filter_sizes=[3, 4, 5], dropout_rate=0.1):
        super(CustomBanglaBertWithCNN, self).__init__()
        self.num_labels = num_labels
        self.filter_sizes = filter_sizes
        self.num_filters = num_filters

        # Load BanglaBERT model and its configuration
        self.config = AutoConfig.from_pretrained(model_name)
        self.config.num_labels = num_labels

        self.bert = AutoModel.from_pretrained(model_name)

        # CNN layers with different filter sizes
        self.convs = nn.ModuleList([
            nn.Conv1d(
                in_channels=self.config.hidden_size,
                out_channels=num_filters,
                kernel_size=filter_size
            ) for filter_size in filter_sizes
        ])

        # Dropout layer
        self.dropout = nn.Dropout(dropout_rate)

        # Classification head
        self.classifier = nn.Linear(len(filter_sizes) * num_filters, num_labels)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
        # Forward pass through BanglaBERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids)

        # Get the sequence output (last hidden states)
        sequence_output = outputs.last_hidden_state  # [batch_size, seq_len, hidden_size]

        # Transpose for Conv1d: [batch_size, hidden_size, seq_len]
        sequence_output = sequence_output.transpose(1, 2)

        # Apply convolution operations
        conv_outputs = []
        for conv in self.convs:
            # Conv1d + ReLU + MaxPooling
            conv_out = torch.relu(conv(sequence_output))  # [batch_size, num_filters, conv_seq_len]
            pooled = torch.max_pool1d(conv_out, kernel_size=conv_out.size(2))  # [batch_size, num_filters, 1]
            conv_outputs.append(pooled.squeeze(2))  # [batch_size, num_filters]

        # Concatenate all conv outputs
        cnn_output = torch.cat(conv_outputs, dim=1)  # [batch_size, len(filter_sizes) * num_filters]

        # Apply dropout
        cnn_output = self.dropout(cnn_output)

        # Classification
        logits = self.classifier(cnn_output)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return {"loss": loss, "logits": logits}

    def save_pretrained(self, output_dir):
        """Save the model weights and configuration."""
        os.makedirs(output_dir, exist_ok=True)
        model_to_save = self.module if hasattr(self, 'module') else self
        torch.save(model_to_save.state_dict(), os.path.join(output_dir, "pytorch_model.bin"))
        self.config.save_pretrained(output_dir)
        print(f"Model saved to {output_dir}")

    @classmethod
    def from_pretrained(cls, pretrained_model_name_or_path, model_name="csebuetnlp/banglabert_large", *model_args, **kwargs):
        """Load a model from a pretrained model."""
        config = AutoConfig.from_pretrained(pretrained_model_name_or_path)
        model = cls(model_name=model_name, num_labels=config.num_labels, **kwargs)
        model.load_state_dict(torch.load(os.path.join(pretrained_model_name_or_path, "pytorch_model.bin")))
        return model

# Load custom model for sequence classification
model = CustomBanglaBertWithCNN(
    model_name=model_path,
    num_labels=2,  # Binary classification
    num_filters=256,
    filter_sizes=[3, 4, 5],
    dropout_rate=0.1
)

# Move model to GPU if available
model.to(device)
print("BanglaBERT Large with CNN model loaded successfully!")

# Test the model with a sample
sentence = "আমি কৃতজ্ঞ কারণ আপনি আমার জন্য অনেক কিছু করেছেন।"
normalized_sentence = normalize(sentence)
inputs = tokenizer(normalized_sentence, return_tensors="pt", padding=True, truncation=True, max_length=128)

# Move inputs to device
inputs = {key: val.to(device) for key, val in inputs.items()}

# Forward pass with dummy labels
labels = torch.tensor([1]).to(device)
outputs = model(**inputs, labels=labels)
print("Model test successful!")
print(f"Sample output: {outputs}")

# Setup optimizer and scheduler
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW

optimizer = AdamW(model.parameters(),
                  lr=2e-5,
                  eps=1e-8)

epochs = 3  # Changed to 3 epochs
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=int(0.1 * total_steps),
                                            num_training_steps=total_steps)

# Training functions
import torch
import numpy as np
from tqdm import tqdm
import time
import os

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def train_epoch(model, train_dataloader, optimizer, scheduler, device):
    model.train()
    total_train_loss = 0
    start_time = time.time()

    for step, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc="Training"):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()

        result = model(b_input_ids,
                       token_type_ids=None,
                       attention_mask=b_input_mask,
                       labels=b_labels)

        loss = result['loss']
        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        if (step + 1) % 500 == 0:
            print(f"Batch {step + 1}/{len(train_dataloader)}: Loss = {loss.item():.4f}")

    avg_train_loss = total_train_loss / len(train_dataloader)
    epoch_time = time.time() - start_time
    print(f"Training completed in {epoch_time // 60:.0f}m {epoch_time % 60:.0f}s")
    return avg_train_loss

def validate_epoch(model, validation_dataloader, device):
    model.eval()
    total_eval_loss = 0
    total_eval_accuracy = 0
    start_time = time.time()

    for batch in tqdm(validation_dataloader, total=len(validation_dataloader), desc="Validating"):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():
            result = model(b_input_ids,
                           token_type_ids=None,
                           attention_mask=b_input_mask,
                           labels=b_labels)

        loss = result['loss']
        logits = result['logits']

        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    validation_time = time.time() - start_time

    print(f"  Accuracy: {avg_val_accuracy:.4f}")
    print(f"  Validation Loss: {avg_val_loss:.4f}")
    print(f"  Validation took: {validation_time // 60:.0f}m {validation_time % 60:.0f}s")
    return avg_val_loss, avg_val_accuracy

def save_best_model(model, tokenizer, output_dir, best_val_loss, avg_val_loss):
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        print(f"  New best validation loss: {best_val_loss:.4f}. Saving model...")

        # Get the model to save
        model_to_save = model.module if hasattr(model, 'module') else model

        # Make all parameters contiguous before saving
        for param in model_to_save.parameters():
            param.data = param.data.contiguous()

        # Save model and tokenizer
        model_to_save.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)

    return best_val_loss

def train_model(model, train_dataloader, validation_dataloader, optimizer, scheduler, tokenizer, epochs, device, output_dir):
    training_stats = []
    best_val_loss = float('inf')

    for epoch_i in range(epochs):
        print(f"\n======== Epoch {epoch_i + 1} / {epochs} ========")
        print("Training...")

        avg_train_loss = train_epoch(model, train_dataloader, optimizer, scheduler, device)
        print(f"\n  Average training loss: {avg_train_loss:.4f}")

        print("\nRunning Validation...")
        avg_val_loss, avg_val_accuracy = validate_epoch(model, validation_dataloader, device)

        training_stats.append({
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
        })

        best_val_loss = save_best_model(model, tokenizer, output_dir, best_val_loss, avg_val_loss)

    print("\nTraining complete!")
    return training_stats

# Train the model
output_dir = '/content/drive/MyDrive/best_model_save_lstm/'
training_stats = train_model(model,
                             train_dataloader,
                             validation_dataloader,
                             optimizer,
                             scheduler,
                             tokenizer,
                             epochs=epochs,
                             device=device,
                             output_dir=output_dir)

# Test evaluation function with comprehensive metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

def predict_and_evaluate(model, test_dataloader, device, dataset_name="Test"):
    model.eval()

    total_test_loss = 0
    total_test_accuracy = 0
    predictions = []
    true_labels = []

    start_time = time.time()

    with torch.no_grad():
        for batch in tqdm(test_dataloader, desc=f"Evaluating {dataset_name}", total=len(test_dataloader)):
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask,
                            labels=b_labels)

            loss = outputs['loss']
            logits = outputs['logits']

            total_test_loss += loss.item()

            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            total_test_accuracy += flat_accuracy(logits, label_ids)

            pred_flat = np.argmax(logits, axis=1).flatten()
            predictions.extend(pred_flat)
            true_labels.extend(label_ids.flatten())

    avg_test_loss = total_test_loss / len(test_dataloader)
    avg_test_accuracy = total_test_accuracy / len(test_dataloader)

    epoch_time = time.time() - start_time
    print(f"{dataset_name} evaluation completed in {epoch_time // 60:.0f}m {epoch_time % 60:.0f}s")

    # Calculate comprehensive metrics
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='weighted')
    recall = recall_score(true_labels, predictions, average='weighted')
    f1 = f1_score(true_labels, predictions, average='weighted')

    # Also calculate metrics for each class
    precision_per_class = precision_score(true_labels, predictions, average=None)
    recall_per_class = recall_score(true_labels, predictions, average=None)
    f1_per_class = f1_score(true_labels, predictions, average=None)

    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'precision_per_class': precision_per_class,
        'recall_per_class': recall_per_class,
        'f1_per_class': f1_per_class,
        'loss': avg_test_loss
    }

    return predictions, true_labels, metrics

def print_comprehensive_metrics(metrics, class_names, dataset_name):
    print(f"\n{'='*50}")
    print(f"{dataset_name.upper()} SET METRICS")
    print(f"{'='*50}")

    print(f"Overall Metrics:")
    print(f"  Accuracy:  {metrics['accuracy']:.4f} ({metrics['accuracy']*100:.2f}%)")
    print(f"  Precision: {metrics['precision']:.4f}")
    print(f"  Recall:    {metrics['recall']:.4f}")
    print(f"  F1-Score:  {metrics['f1_score']:.4f}")
    print(f"  Loss:      {metrics['loss']:.4f}")

    print(f"\nPer-Class Metrics:")
    for i, class_name in enumerate(class_names):
        print(f"  {class_name}:")
        print(f"    Precision: {metrics['precision_per_class'][i]:.4f}")
        print(f"    Recall:    {metrics['recall_per_class'][i]:.4f}")
        print(f"    F1-Score:  {metrics['f1_per_class'][i]:.4f}")

# Evaluate on test set
print("\n" + "="*50)
print("FINAL EVALUATION ON TEST SET")
print("="*50)

test_predictions, test_true_labels, test_metrics = predict_and_evaluate(
    model, test_dataloader, device, "Test Set"
)

# Also evaluate on validation set for comparison
print("\nValidation Set Results (for comparison):")
val_predictions, val_true_labels, val_metrics = predict_and_evaluate(
    model, validation_dataloader, device, "Validation Set"
)

# Updated class names - Changed from No Sarcasm/Sarcasm to Positive/Negative
class_names = ['Positive', 'Negative']

# Print comprehensive metrics
print_comprehensive_metrics(test_metrics, class_names, "Test")
print_comprehensive_metrics(val_metrics, class_names, "Validation")

# Enhanced Confusion Matrix and Classification Report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt

def plot_enhanced_confusion_matrix_and_report(predictions, true_labels, class_names, dataset_name, metrics):
    # Confusion Matrix
    cm = confusion_matrix(true_labels, predictions)

    fig, axes = plt.subplots(1, 2, figsize=(15, 6))

    # Plot confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(cmap='Blues', ax=axes[0], values_format='d')
    axes[0].set_title(f"Confusion Matrix - {dataset_name}")

    # Plot normalized confusion matrix
    cm_normalized = confusion_matrix(true_labels, predictions, normalize='true')
    disp_norm = ConfusionMatrixDisplay(confusion_matrix=cm_normalized, display_labels=class_names)
    disp_norm.plot(cmap='Blues', ax=axes[1], values_format='.2f')
    axes[1].set_title(f"Normalized Confusion Matrix - {dataset_name}")

    plt.tight_layout()
    plt.show()

    # Detailed classification report
    report = classification_report(true_labels, predictions, target_names=class_names, digits=4)
    print(f"\nDetailed Classification Report - {dataset_name}:")
    print("="*80)
    print(report)

    # Additional metrics summary
    print(f"\nMetrics Summary - {dataset_name}:")
    print("="*50)
    print(f"Overall Accuracy: {metrics['accuracy']:.4f}")
    print(f"Macro Avg Precision: {np.mean(metrics['precision_per_class']):.4f}")
    print(f"Macro Avg Recall: {np.mean(metrics['recall_per_class']):.4f}")
    print(f"Macro Avg F1-Score: {np.mean(metrics['f1_per_class']):.4f}")
    print(f"Weighted Avg Precision: {metrics['precision']:.4f}")
    print(f"Weighted Avg Recall: {metrics['recall']:.4f}")
    print(f"Weighted Avg F1-Score: {metrics['f1_score']:.4f}")

# Plot enhanced results for test set
print("\n" + "="*30 + " TEST SET DETAILED RESULTS " + "="*30)
plot_enhanced_confusion_matrix_and_report(test_predictions, test_true_labels, class_names, "Test Set", test_metrics)

# Plot enhanced results for validation set
print("\n" + "="*30 + " VALIDATION SET DETAILED RESULTS " + "="*30)
plot_enhanced_confusion_matrix_and_report(val_predictions, val_true_labels, class_names, "Validation Set", val_metrics)

# Enhanced training statistics with metrics comparison
print("\n" + "="*30 + " TRAINING STATISTICS " + "="*30)
training_df = pd.DataFrame(training_stats)
print(training_df)

# Create a comprehensive metrics comparison table
metrics_comparison = pd.DataFrame({
    'Dataset': ['Validation', 'Test'],
    'Accuracy': [val_metrics['accuracy'], test_metrics['accuracy']],
    'Precision': [val_metrics['precision'], test_metrics['precision']],
    'Recall': [val_metrics['recall'], test_metrics['recall']],
    'F1-Score': [val_metrics['f1_score'], test_metrics['f1_score']],
    'Loss': [val_metrics['loss'], test_metrics['loss']]
})

print("\n" + "="*30 + " METRICS COMPARISON " + "="*30)
print(metrics_comparison.round(4))

# Enhanced training progress plot
plt.figure(figsize=(15, 10))

# Training and validation loss
plt.subplot(2, 2, 1)
plt.plot(training_df['epoch'], training_df['Training Loss'], 'b-o', label='Training Loss', linewidth=2)
plt.plot(training_df['epoch'], training_df['Valid. Loss'], 'r-o', label='Validation Loss', linewidth=2)
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

# Validation accuracy
plt.subplot(2, 2, 2)
plt.plot(training_df['epoch'], training_df['Valid. Accur.'], 'g-o', label='Validation Accuracy', linewidth=2)
plt.title('Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

# Metrics comparison bar chart
plt.subplot(2, 2, 3)
metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
val_scores = [val_metrics['accuracy'], val_metrics['precision'], val_metrics['recall'], val_metrics['f1_score']]
test_scores = [test_metrics['accuracy'], test_metrics['precision'], test_metrics['recall'], test_metrics['f1_score']]

x = np.arange(len(metrics_names))
width = 0.35

plt.bar(x - width/2, val_scores, width, label='Validation', alpha=0.8)
plt.bar(x + width/2, test_scores, width, label='Test', alpha=0.8)
plt.xlabel('Metrics')
plt.ylabel('Score')
plt.title('Validation vs Test Metrics Comparison')
plt.xticks(x, metrics_names, rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)

# Per-class metrics comparison
plt.subplot(2, 2, 4)
class_metrics = ['Precision', 'Recall', 'F1-Score']
positive_scores = [test_metrics['precision_per_class'][0], test_metrics['recall_per_class'][0], test_metrics['f1_per_class'][0]]
negative_scores = [test_metrics['precision_per_class'][1], test_metrics['recall_per_class'][1], test_metrics['f1_per_class'][1]]

x = np.arange(len(class_metrics))
plt.bar(x - width/2, positive_scores, width, label='Positive', alpha=0.8)
plt.bar(x + width/2, negative_scores, width, label='Negative', alpha=0.8)
plt.xlabel('Metrics')
plt.ylabel('Score')
plt.title('Per-Class Metrics (Test Set)')
plt.xticks(x, class_metrics)
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Create a final summary table
final_results = {
    'Model': ['BanglaBERT + CNN'],
    'Dataset': ['Test Set'],
    'Accuracy (%)': [f"{test_metrics['accuracy']*100:.2f}%"],
    'Precision': [f"{test_metrics['precision']:.4f}"],
    'Recall': [f"{test_metrics['recall']:.4f}"],
    'F1-Score': [f"{test_metrics['f1_score']:.4f}"],
    'Loss': [f"{test_metrics['loss']:.4f}"]
}

final_df = pd.DataFrame(final_results)

print("\n" + "="*80)
print("FINAL MODEL PERFORMANCE SUMMARY")
print("="*80)
print(final_df.to_string(index=False))

# Per-class performance summary
per_class_results = pd.DataFrame({
    'Class': class_names,
    'Precision': [f"{score:.4f}" for score in test_metrics['precision_per_class']],
    'Recall': [f"{score:.4f}" for score in test_metrics['recall_per_class']],
    'F1-Score': [f"{score:.4f}" for score in test_metrics['f1_per_class']]
})

print(f"\nPER-CLASS PERFORMANCE (TEST SET):")
print("-" * 50)
print(per_class_results.to_string(index=False))

print("\n" + "="*80)
print("TRAINING AND EVALUATION COMPLETED!")
print("="*80)
print(f"Final Test Accuracy: {test_metrics['accuracy']:.4f} ({test_metrics['accuracy']*100:.2f}%)")
print(f"Final Test Precision: {test_metrics['precision']:.4f}")
print(f"Final Test Recall: {test_metrics['recall']:.4f}")
print(f"Final Test F1-Score: {test_metrics['f1_score']:.4f}")
print(f"Model saved to: {output_dir}")
print("="*80)

# Save metrics to CSV files for future reference
training_df.to_csv('/content/drive/MyDrive/training_statistics.csv', index=False)
metrics_comparison.to_csv('/content/drive/MyDrive/metrics_comparison.csv', index=False)
final_df.to_csv('/content/drive/MyDrive/final_results.csv', index=False)
per_class_results.to_csv('/content/drive/MyDrive/per_class_results.csv', index=False)

print(f"\nResults saved to CSV files:")
print("- Training statistics: /content/drive/MyDrive/training_statistics.csv")
print("- Metrics comparison: /content/drive/MyDrive/metrics_comparison.csv")
print("- Final results: /content/drive/MyDrive/final_results.csv")
print("- Per-class results: /content/drive/MyDrive/per_class_results.csv")